In [94]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers, Model
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.utils import class_weight
import datetime

In [95]:
#model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = VGG16(include_top=True)

last_layer = model.get_layer('fc1')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

FEATURE_SIZE = 2048
reduction = 2048

last layer output shape: (None, 4096)


In [96]:
for layer in model.layers:
    layer._name = layer._name + str('_C')

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
cut_model = Model(model.input, x)

In [97]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(cut_model.predict(x))
    return features

In [98]:
%%time
#print(decode_predictions(get_features('00000')))


Wall time: 495 µs


In [99]:
%%time
data = np.zeros((10000,4096 + 1))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

#data = features()

Wall time: 39.7 ms


In [100]:
#save_ass = 'features_vgg16.csv'

In [101]:
print(np.count_nonzero(data))
save_ass = 'features_vgg16.csv'

#np.savetxt(save_ass, data, delimiter=',')

9999


using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [102]:
#lets prepare the data for the NN

In [103]:
%%time

ABC1 = np.loadtxt("train_triplets.txt").astype('int') 

data = np.genfromtxt(save_ass, delimiter= ',')

svd = PCA(n_components=reduction)
svd.fit(data)
transformed = svd.transform(data)
print(transformed.shape)
np.savetxt("transformed_2048_pca_vgg16.csv", transformed, delimiter=',')

def load_features(int_id):
    return data[int_id]

(10000, 2048)
Wall time: 1min 23s


ABC1 = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt("transformed_2k_vgg16.csv", delimiter= ',')
data = np.array(data)
#data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

In [105]:
data1 = np.genfromtxt("transformed_2k_vgg16.csv", delimiter= ',')
data2 = np.genfromtxt("transformed_2048_pca_vgg16.csv", delimiter= ',')
data3 = np.genfromtxt("transformed_2048_vgg16.csv", delimiter= ',')

In [106]:
print(np.max(data1))
print(np.min(data1))
print(np.count_nonzero(data1>0)/10000)
print(data1)

print(np.max(data2))
print(np.min(data2))
print(np.count_nonzero(data2>0)/10000)
print(data2)

print(np.max(data3))
print(np.min(data3))
print(np.count_nonzero(data3>0)/10000)
print(data3)

437.1729314338028
-232.46749067398565
1023.6355
[[ 1.13651419e+02 -7.90404109e+00  1.45638476e+01 ...  2.45957482e-02
  -3.80949972e+00  1.88267921e+00]
 [ 1.63260723e+02 -2.88156321e+01  4.03460357e+00 ...  1.59725514e+00
   8.89895754e-01 -1.33387738e-01]
 [ 1.20817501e+02  4.81291883e+01  3.80443678e+01 ... -2.35474535e+00
  -5.82574743e-01  1.57955269e+00]
 ...
 [ 1.54377886e+02 -4.68704584e+00 -2.45622575e+00 ...  8.00527003e-01
   9.71803475e-01  1.89183454e+00]
 [ 1.14112921e+02 -2.61596385e+01  7.62224940e+01 ...  1.08829451e+00
  -4.25203554e-01 -2.99227767e+00]
 [ 1.15845917e+02  7.02213942e+01  5.27495035e+01 ... -7.76336616e-01
   1.78615267e+00  2.14172536e+00]]
4999.509837181176
-4999.495408342369
1023.455
[[ 4.99950984e+03 -1.06269086e+01 -5.11564339e+01 ...  7.74110307e-01
  -6.25773662e-01 -4.44572817e+00]
 [ 4.99849075e+03  3.11837707e+01 -2.76554560e+00 ...  2.11490466e+00
  -1.26303460e-01 -9.46173446e-01]
 [ 4.99748581e+03 -6.21577864e+01 -4.08563130e+01 ...  6.701

In [ ]:
%%time

# There are 4 different permutations of ABC that we can know the answer of. (CAB and CBA we don't know)
ONE_SET_SIZE = 59515

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((ONE_SET_SIZE * 2,1))
data_y0 = np.zeros((ONE_SET_SIZE * 2,1))

#create the inverse dataset and a set where B is connected to A (and inv)

ACB0 = np.zeros((ONE_SET_SIZE,3)).astype('int')    # One needs 3 different assignments since the arrays would be linked otherwise.
BAC1 = np.zeros((ONE_SET_SIZE,3)).astype('int')
BCA0 = np.zeros((ONE_SET_SIZE,3)).astype('int')

ACB0[:,0] = ABC1[:,0]
BAC1[:,1] = ABC1[:,0]
BCA0[:,2] = ABC1[:,0]

ACB0[:,2] = ABC1[:,1]
BAC1[:,0] = ABC1[:,1]
BCA0[:,0] = ABC1[:,1]

ACB0[:,1] = ABC1[:,2]
BAC1[:,2] = ABC1[:,2]
BCA0[:,1] = ABC1[:,2]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((ABC1, BAC1, ACB0, BCA0),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((ONE_SET_SIZE * 4,FEATURE_SIZE * 3))

for i in range(0,3):
        for j in range(0,ONE_SET_SIZE * 4):
            data_nn_x[j,range(i*FEATURE_SIZE, FEATURE_SIZE*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%50000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

In [ ]:
#np.savetxt("data_nn_x.csv", data_nn_x, delimiter=',')
data_nn_x.astype('float16')

In [ ]:
%%time
collab = np.concatenate((data_nn_x,labels), axis=1, dtype='float16')
np.random.shuffle(collab)

data_x = collab[:,:FEATURE_SIZE * 3]
data_y = collab[:,FEATURE_SIZE * 3]

In [ ]:
#print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

In [ ]:
%xdel data_nn_x
print(data_x.dtype)
print(data_y.dtype)

print(np.max(data_x))
print(np.min(data_x))

In [ ]:
%%time
#init=uniform
#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(4096, activation='relu', input_dim=FEATURE_SIZE * 3, kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.5))
#neuralNetwork.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.003)))
#neuralNetwork.add(BatchNormalization())
#neuralNetwork.add(Dropout(0.4))
neuralNetwork.add(Dense(256, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

neuralNetwork.summary()

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_x.astype('float16'), data_y.astype('float16'), epochs=128, batch_size=1024, verbose=2, validation_split = 0.2)

#evaluation of the network prediction
#predict_test_nn = neuralNetwork.predict(df_test_onehot_nn)
#predict_train_nn = neuralNetwork.predict(df_train_onehot_nn)
#print(predict_test_nn)
#print(predict_train_nn)
#predict_test_nn = (predict_test_nn >= 0.5)

Layers:                         F.      Epoch   Val_Acc
4096-4096-1024-256-64-1         2048    64      0.7962
4096-4096-1024-256-64-1         2048    128     0.8043
6144-4096-2048-1024-256-64-1    2048    64      0.7943
6144-4096-2048-1024-256-64-1    2048    256     0.8044
4096-4096-1024-256-64-1         2048    64      0.8329 #using 0.1 initial dropout (score was super bad)
4096-4096-1024-256-64-1         2048    64      0.7961 #0.5 Drop except for last 0.2
6144-1024-256-64-1              2048    128     0.8031 #check the SVD value and retry again
8162-256-64-1                   3072    256     0.732
4096-256-64-1                   1024    128     0.718
4096-256-64-1                   2048    64      0.804
4096-256-64-1                   2560    64

Lets try to find out what dim-red. is needed: 1024-0.5-64-0.2-1 32e
Factor:         Val_Acc:
3072            0.725
2048            0.796 #wtf is this even.
1024            0.712

In [ ]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,FEATURE_SIZE * 3))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*FEATURE_SIZE , FEATURE_SIZE*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

In [ ]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_retrain_v8.txt",np.round(export,decimals=0), fmt='%i')